<a href="https://colab.research.google.com/github/isold23/RAG/blob/main/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-cpp-python langchain unstructured[pdf]

In [3]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain


llm = LlamaCpp(
    model_path="/content/drive/MyDrive/chinese-llama-2-7b.Q2_K.gguf",
    input={"temperature": 1, "top_p": 1},
    n_ctx=2048,
    verbose=True,
)

from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import urllib
from time import monotonic

url = "http://www.hzdianyue.com/doc/readme.txt"

article = urllib.request.urlopen(url).read().decode("utf-8")

text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(article)

docs = [Document(page_content=t) for t in texts]

template = """
SYSTEM: Você é um especialista em resumir textos.
USER: Escreva um resumo sucinto do seguinte texto: {text}
ASSISTANT:
"""
prompt = PromptTemplate(template=template, input_variables=["text"])

chain = LLMChain(prompt=prompt, llm=llm)

chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

start_time = monotonic()
summary = chain.run(docs)
print(f"Run time {monotonic() - start_time} seconds")

print(summary)
print(len(article))
print(len(summary))

/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /content/drive/MyDrive/chinese-llama-2-7b.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 

Run time 462.23197194399927 seconds
1. 在客户咨询或问题的描述中，找到一个最能体现其服务能力的典型案例。2. 如果您在回答客户咨询或问题时需要使用示例文本，您可以创建一个示例文本。3. 在客户咨询或问题的描述中，找到一个最能体现其服务能力的典型案例。4. 如果您在回答客户咨询或问题时需要使用示例文本，您可以创建一个示例文本。1. 在客户咨询或问题的描述中，找到一个最能体现其服务能力的典型案例。2. 如果您在回答客户咨询或问题时需要使用示例文本，您可以创建一个示例文本。3. 如果您在回答客户咨询或问题时需要使用示例文本，您可以创建一个示例文本。1. 如果您需要展示您的服务能力，您可以创建一个示例文本。2. 如果您在回答客户咨询或问题时需要使用示例文本，您可以创建一个示例文本。3. 如果您需要展示您的服务能力，您可以创建一个示例文本。1. 如果您需要展示您的服务能力，您可以创建一个示例文本。2. 如果您在
1566
389
